#Install

In [ ]:
!pip install python-pptx

from google.colab import drive
import os
import pptx
from pptx.enum.shapes import MSO_SHAPE_TYPE

!pip install openai
import openai
import os
import time
import pandas as pd
!pip install --upgrade openai

In [ ]:
#your API key
os.environ["OPENAI_API_KEY"] = '[YOUR_API_KEY]'

#load ppt from the google drive
drive.mount('/content/gdrive')
os.chdir("/content/gdrive/My Drive/analyzer/input_ppt")

#Functions

In [ ]:
class ChatApp: #no demo, only template
    def __init__(self, input):
        # Setting the API key to use the OpenAI API
        openai.api_key = os.getenv("OPENAI_API_KEY")
        self.messages = input

    def chat(self, message):
        self.messages.append({"role": "user", "content": message})
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=self.messages,
            temperature=0.7,
            top_p = 1,
        )
        self.messages.append({"role": "assistant", "content": response["choices"][0]["message"].content})

        for i in range(2):
          self.messages.pop()

        return response["choices"][0]["message"]["content"]

In [ ]:
def make_prompt_format(system, user, assistant): #prompt demonstration
  #system
  input = [{"role": "system", "content": system}]
  if user != "":
    #user
    demo_user = []
    demo_user.append(user)
    #assistant
    demo_assistant = []
    demo_assistant.append(assistant)

    #make format for API
    demo = pd.DataFrame({'user': demo_user, 'assistant': demo_assistant})
    for prompt, completion in zip(demo['user'], demo['assistant']):
      message_dict = {"role": "user", "content": prompt}
      input.append(message_dict)
      message_dict = {"role": "assistant", "content": completion}
      input.append(message_dict)
    
  return input

#Main

##Load Data

In [ ]:
#load ppt file
ppt = pptx.Presentation('Architecture_Design.pptx')

#for full slide
full_slide = []
content_full = "Entire contents: "
for i, slide in enumerate(ppt.slides):
  content_full = content_full + f"  *slide {i}: "
  for shape in slide.shapes:
    if hasattr(shape, "text"):
      content_full = content_full + shape.text
full_slide = content_full

# print("*"*10)
# print(full_slide)

#for each slide, keywords
each_slide = {} #dictionary form
for i, slide in enumerate(ppt.slides):
  content_each = ""
  for shape in slide.shapes:
    if hasattr(shape, "text"):
      content_each = content_each + shape.text
  each_slide[f'{i}'] = content_each 

# print("*"*10)
# print(each_slide)

##Keyword

In [ ]:
# Extract Keywords from each slide with the explanation

In [ ]:
#prompt demonstration
keyword_system = "I want you to act as a keyword detector. I will give you the contents of a PowerPoint presentation, and you will provide me with the relevant keywords along with their explanations."
keyword_user = "slide 16: Agility & Architecture Agile approach implements a few requirements in each sprint. However, we cannot do the same for the architecture of a complex system! We cannot start with the bare-minimum architecture & keep expanding.  Refactoring/ revamping the architecture may be as bad as restarting from scratch – can be expensive & time-consuming!  In other words, agile approach is primarily for development & beyond…"
keyword_assistant = "slide: 16: 1. Agility: the ability of a team or organization to quickly adapt to changing circumstances, requirements, or priorities. It focuses on being flexible, responsive, and adaptive in the face of uncertainties, evolving customer needs, and market conditions. 2. Agile approach: a project management and product development methodology that emphasizes iterative and incremental progress, collaboration, and flexibility. In Agile, work is organized into small, manageable tasks called sprints, which are typically two to four weeks in duration. Teams prioritize and complete these tasks while regularly reassessing and adjusting their priorities based on feedback and changing requirements. 3. Bare-minimum architecture: the simplest possible structure or design for a system that can still meet its basic functional requirements. This approach is usually not recommended for complex systems, as it can lead to significant challenges when it comes to expanding or refining the architecture to accommodate new features or improvements. 4. Refactoring: the process of restructuring and improving existing code without changing its external behavior or functionality. This can involve cleaning up the code, optimizing performance, or simplifying the design to make it more maintainable and easier to understand. 5. Revamping: the process of making significant improvements or overhauling an existing system, product, or process. In the context of software architecture, this might involve rethinking the overall design, updating outdated components, or introducing new technologies to better align with current requirements and best practices. 6. Scratch: In the context of software development, \"starting from scratch\" refers to the act of creating a new system, application, or component entirely from the beginning, rather than building upon or modifying an existing one. This can be a time-consuming and costly process, but in some cases, it may be necessary if the existing system or architecture is deemed unsuitable or too difficult to refactor or revamp."
prompt_keyword = make_prompt_format(keyword_system, keyword_user, keyword_assistant)

#extract keyword for each slide with an explanation
result_keyword = {}
for i in range(len(each_slide)):
  app_keyword = ChatApp(prompt_keyword)
  message = each_slide[f'{i}']
  res = app_keyword.chat(message)
  time.sleep(3) #API call restriction, 20 calls per a minute
  result_keyword[f'{i}'] = res

result_keyword